In [1]:
import sys
import os

# Path to the folder containing ipyformkit (one level up from examples/)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")
# Try importing now
import ipyformkit as ifk

Project root added to sys.path: /Users/hoppe/Documents/Notebooks/IpyFormKit


<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML

test = widgets.Button(description='Click me!')
test.add_class('text')
display(test)

Button(description='Click me!', style=ButtonStyle(), _dom_classes=('text',))

In [3]:
import ipyformkit as ifk

test = {'first name':'John',
        'last name':'Doe',
        ('age', 'gender'):(-42, ('male', 'female', 'non-binary')),
        'height':1.82,
        'date of birth':'01.01.1980',
        ('input 1', 'input 2', 'input 3', 'input 4', 'input 5'):(1,2,3,4,5),
        ('street', 'house'):('Baker Street', 42),
        'additional info':{'hobbies':'reading, running...',},
        'submit':None,
       }

mandatory = {'first name': True, 'last name': True, 'age':True}
disable = {'last name': lambda d: d['first name'] == '',}
hide = {'house': lambda d: d['street'] == '',}
check = {'age': lambda d: d['age'] > 0,}

form = ifk.Form(test, title='Test Form', mandatory=mandatory, disable=disable, hide=hide, check=check)
form.display()

In [4]:
form.check_and_return_values()

Mandatory field 'first name' is empty.
Mandatory field 'last name' is empty.
Invalid input in field 'age'.


In [5]:
form.check_conditions[form.widgets_dict['age']]

<function __main__.<lambda>(d)>

In [6]:
form.widgets_dict

{'first name': VBox(children=(Label(value='first name', _dom_classes=('ifk-widget-label',)), Text(value='', placeholder='John', _dom_classes=('ifk-widget-input', 'ifk-widget-input-missing'))), _dom_classes=('ifk-widget-box', 'ifk-widget-Text')),
 'last name': VBox(children=(Label(value='last name', _dom_classes=('ifk-widget-label',)), Text(value='', disabled=True, placeholder='Doe', _dom_classes=('ifk-widget-input', 'ifk-widget-input-missing'))), _dom_classes=('ifk-widget-box', 'ifk-widget-Text')),
 'age': VBox(children=(Label(value='age', _dom_classes=('ifk-widget-label',)), IntText(value=-42, _dom_classes=('ifk-widget-input', 'ifk-widget-input-error'))), _dom_classes=('ifk-widget-box', 'ifk-widget-IntText')),
 'gender': VBox(children=(Label(value='gender', _dom_classes=('ifk-widget-label',)), Dropdown(_dom_classes=('ifk-widget-input',), options=('male', 'female', 'non-binary'), value='male')), _dom_classes=('ifk-widget-box', 'ifk-widget-Dropdown')),
 'height': VBox(children=(Label(va

In [7]:
import ipyformkit as ifk

example_data = {
    'username': 'user123',
    'password': '',
    'confirm password': '',
    'email': 'user@example.com',
    'subscribe to newsletter': True,
    'newsletter frequency': ('Daily', 'Weekly', 'Monthly'),
    'address': '',
    ('city', 'state', 'zip code'): ('', '', ''),
    'additional info': {'comments': 'Enter your comments here...'},
}

disable_conditions = {
    'confirm password': lambda d: d['password'] == '',
    'newsletter frequency': lambda d: not d['subscribe to newsletter'],
}

hide_conditions = {
    'city': lambda d: d['address'] == '',
    'state': lambda d: d['address'] == '',
    'zip code': lambda d: d['address'] == '',
}

form = ifk.Form(
    example_data,
    title='User Registration Form',
    mandatory={'username': True, 'password': True, 'email': True},
    disable=disable_conditions,
    hide=hide_conditions,
)

form.display()

In [8]:
btn = widgets.Text(description='Submit')
display(btn)

Text(value='', description='Submit')